In [1]:
import folium

map_y = folium.Map([37.52860, 126.93431],zoom_start=15)
map_y

In [2]:
lat1,long1 = 37.52860, 126.93431 #여의도 한강공원
lat2,long2 = 37.52400, 126.91889 #여의도공원
lat3,long3 = 37.51864, 126.92041 #샛강생태공원

map_y = folium.Map([lat1, long1],zoom_start=15)
folium.Marker([lat1, long1],tooltip='여의도 한강공원',
             icon=folium.Icon(color='red',icon='automobile',prefix='fa')).add_to(map_y)
folium.Marker([lat2, long2],tooltip='여의도공원',
             icon=folium.Icon(color='blue',icon='balance-scale',prefix='fa')).add_to(map_y)
folium.Marker([lat3, long3],tooltip='샛강생태공원',
             icon=folium.Icon(color='purple',icon='ban',prefix='fa')).add_to(map_y)
map_y

In [32]:
import folium

lat = [37.52860, 37.52400, 37.51865]
long = [126.93431, 126.91889, 126.92041]
names = ['여의도 한강공원','여의도공원','샛강생태공원']
icons = ['automobile','balance-scale','ban']
colors = ['red','blue','purple']

map_y = folium.Map([lat[0],long[0]],zoom_start=15)

for i in range(len(lat)):
    folium.Marker([lat[i],long[i]], tooltip=names[i], icon=folium.Icon(color=colors[i],icon=icons[i],prefix='fa')).add_to(map_y)
    
map_y

In [37]:
#필요한 데이터 파일 모두 읽어 들이기
import csv
import operator
import folium

#인구 데이터 읽어와서 리스트로 저장하기
f1 = open('LOCAL_PEOPLE_DONG_202112.csv',encoding='utf8')
data = csv.reader(f1)
next(data)
data=list(data)

#행정동코드 데이터 읽어와서 리스트로 저장하기
f2 = open('dong_code.csv',encoding='utf8')
code_data = csv.reader(f2)
next(code_data)
next(code_data)
code_data=list(code_data)

#위치 정보 데이터 읽어와서 리스트로 저장하기
f3 = open('dong_location.csv')
location_data = csv.reader(f3)
next(location_data)
location_data = list(location_data)

#인구 데이터의 데이터 타입 변환하기
for row in data:
    for i in range(1,32):
        if i<=2:
            row[i]=int(row[i])
        else:
            row[i]=float(row[i])

#행정동코드 데이터의 데이터 타입 변환하기
for row in code_data:
    row[0], row[1] = int(row[0]), int(row[1])

#위치 정보 데이터의 데이터 타입 변환하기
for row in location_data:
    row[2],row[-2],row[-1] = int(row[2]),float(row[-2]),float(row[-1])
    
"""행정동별 인구수를 저장할 수 있게 빈 딕셔너리를 만들고
행정동별 총생활인구를 딕셔너리에 저장하기"""
dong_population = {}

for row in data:
    dong_code, p = row[2],row[3]
    if dong_code not in dong_population.keys():
        dong_population[dong_code] = p
    else:
        dong_population[dong_code]+=p
        
#인구수를 기준으로 내림차순 정렬하고, 앞에서 10개 항목만 별도로 저장하기

top10 = sorted(dong_population.items(), key=operator.itemgetter(1), reverse=True)[:10]

top10_code8=[]
for t in top10:
    top10_code8.append(t[0])

top10_name = [0 for i in range(10)]
top10_lat = [0 for i in range(10)]
top10_long = [0 for i in range(10)]

#8자리 행정동코드에 해당하는 행정동명 찾기
for i in range(len(top10_code8)):
    for row in code_data:
        if row[1] == top10_code8[i]:
            top10_name[i] = row[-1] #행정동명 리스트에 저장하기
    #행정동명에 해당하는 위도와 경도를 찾아 리스트에 저장하기
    for row in location_data:
        if row[3] == top10_name[i]:
            top10_lat[i] = row[-1]
            top10_long[i] = row[-2]

#행정동명으로 위치 정보를 찾지 못하면 7자리 행정동코드로 찾기            
for i in range(len(top10_lat)):
    #위도나 경도가 0인 요소가 있다면
    if top10_lat[i] == 0 or top10_long[i]==0:
        #code_data에서 7자리 행정동코드를 구해 dong7에 저장하기
        for row in code_data:
            if top10_code8[i] == row[1]:
                dong7 = row[0]
        #location_data에서 7자리 행정동코드가 dong7인 행을 찾아 위도와 경도 구하기
        for row in location_data:
            if dong7 == row[2]:
                top10_lat[i], top10_long[i] = row[-1],row[-2]

#완성된 리스트로 지도 그리기
#행정동명 앞에 순위 표시하기
for i in range(len(top10_name)):
    top10_name[i] = str(i+1) + '. ' + top10_name[i]
    
map_top10 = folium.Map([top10_lat[0], top10_long[0]],zoom_start=11)
for i in range(10):
    folium.Marker([top10_lat[i], top10_long[i]], tooltip=top10_name[i],
    icon=folium.Icon(color='red', icon='star')).add_to(map_top10)

map_top10

In [38]:
"""인구 데이터를 읽으며 행정동별 남성 인구수를 저장하는 딕셔너리"""
dong_population = {}

for row in data:
    dong_code, p = row[2],sum(row[4:18])
    if dong_code not in dong_population.keys():
        dong_population[dong_code] = p
    else:
        dong_population[dong_code]+=p
        
#인구수를 기준으로 내림차순 정렬하고, 앞에서 10개 항목만 별도로 저장하기

top10 = sorted(dong_population.items(), key=operator.itemgetter(1), reverse=True)[:10]

top10_code8=[]
for t in top10:
    top10_code8.append(t[0])

top10_name = [0 for i in range(10)]
top10_lat = [0 for i in range(10)]
top10_long = [0 for i in range(10)]

#8자리 행정동코드에 해당하는 행정동명 찾기
for i in range(len(top10_code8)):
    for row in code_data:
        if row[1] == top10_code8[i]:
            top10_name[i] = row[-1] #행정동명 리스트에 저장하기
    #행정동명에 해당하는 위도와 경도를 찾아 리스트에 저장하기
    for row in location_data:
        if row[3] == top10_name[i]:
            top10_lat[i] = row[-1]
            top10_long[i] = row[-2]

#행정동명으로 위치 정보를 찾지 못하면 7자리 행정동코드로 찾기            
for i in range(len(top10_lat)):
    #위도나 경도가 0인 요소가 있다면
    if top10_lat[i] == 0 or top10_long[i]==0:
        #code_data에서 7자리 행정동코드를 구해 dong7에 저장하기
        for row in code_data:
            if top10_code8[i] == row[1]:
                dong7 = row[0]
        #location_data에서 7자리 행정동코드가 dong7인 행을 찾아 위도와 경도 구하기
        for row in location_data:
            if dong7 == row[2]:
                top10_lat[i], top10_long[i] = row[-1],row[-2]

#완성된 리스트로 지도 그리기
#행정동명 앞에 순위 표시하기
for i in range(len(top10_name)):
    top10_name[i] = str(i+1) + '. ' + top10_name[i]
    
map_top10 = folium.Map([top10_lat[0], top10_long[0]],zoom_start=11)
for i in range(10):
    if i<5:
        folium.Marker([top10_lat[i], top10_long[i]], tooltip=top10_name[i],
        icon=folium.Icon(color='darkblue', icon='male',prefix='fa')).add_to(map_top10)
    else:
        folium.Marker([top10_lat[i], top10_long[i]], tooltip=top10_name[i],
        icon=folium.Icon(color='lightblue', icon='male',prefix='fa')).add_to(map_top10)

map_top10

In [39]:
for i in range(len(top10_name)):
    top10_name[i] = str(i+1) + '. ' + top10_name[i]
    
map_top10 = folium.Map([top10_lat[0], top10_long[0]],zoom_start=11)
for i in range(10):
    if i<5:
        folium.Marker([top10_lat[i], top10_long[i]], tooltip=top10_name[i],
        icon=folium.Icon(color='red', icon='female',prefix='fa')).add_to(map_top10)
    else:
        folium.Marker([top10_lat[i], top10_long[i]], tooltip=top10_name[i],
        icon=folium.Icon(color='pink', icon='female',prefix='fa')).add_to(map_top10)

map_top10

In [42]:
import folium

lat, long = 35.15870, 129.16047
hmap = folium.Map([lat, long],zoom_start=11)

folium.Marker([lat, long], tooltip='해운대',
        icon=folium.Icon(color='red', icon='hand-o-down',prefix='fa')).add_to(hmap)

hmap

In [44]:
import folium

lat = [35.15870, 35.23360, 35.17357]
long = [129.16047, 129.08309, 128.94632]
name = ['부산대 해수욕장', '부산대학교','김해국제공항']

hmap = folium.Map([lat[0], long[0]],zoom_start=11)

for i in range(len(lat)):
    folium.Marker([lat[i],long[i]], tooltip=name[i], 
                  icon=folium.Icon(color='red', icon='hand-o-down',
                                  prefix='fa')).add_to(hmap)

hmap